In [6]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 0s (1,221 kB/s)     
Selecting previously unselected package poppler-utils.
(Reading database ... 128663 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
pip install pdf2image typhoon_ocr PyPDF2

In [3]:
import os
max_workers = os.cpu_count() or 1
max_workers

4

In [ ]:
import os
import pandas as pd
from PIL import Image
from io import BytesIO
import base64
import torch
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration
from typhoon_ocr.ocr_utils import get_anchor_text, render_pdf_to_base64png
from PyPDF2 import PdfReader
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

pdf_folder = "/kaggle/input/datasetaaaaa"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "scb10x/typhoon-ocr-3b", torch_dtype=torch.bfloat16
).half().eval().to(device)

processor = AutoProcessor.from_pretrained("scb10x/typhoon-ocr-3b")

def make_prompt(base_text):
    return (
        f"Below is an image of a document page along with its dimensions. "
        f"Simply return the markdown representation of this document, presenting tables in markdown format as they naturally appear.\n"
        f"If the document contains images, use a placeholder like dummy.png for each image.\n"
        f"Your final output must be in JSON format with a single key `natural_text` containing the response.\n"
        f"RAW_TEXT_START\n{base_text}\nRAW_TEXT_END"
    )

def process_page(filename, full_path, page_num):
    try:
        base_text = get_anchor_text(full_path, page_num, pdf_engine="pdfreport", target_length=1500)
        image_base64 = render_pdf_to_base64png(full_path, page_num, target_longest_image_dim=800)

        prompt_text = make_prompt(base_text)

        messages = [{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt_text},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_base64}"}},
            ]
        }]

        main_image = Image.open(BytesIO(base64.b64decode(image_base64)))
        text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = processor(
            text=[text],
            images=[main_image],
            padding=True,
            return_tensors="pt",
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.inference_mode():
            output = model.generate(
                **inputs,
                temperature=0.1,
                max_new_tokens=1500,
                num_return_sequences=1,
                repetition_penalty=1.2,
                do_sample=False,
            )

        prompt_length = inputs["input_ids"].shape[1]
        new_tokens = output[:, prompt_length:]
        text_output = processor.tokenizer.batch_decode(new_tokens, skip_special_tokens=True)[0]

        return {"filename": filename, "page": page_num, "text_ocr": text_output}
    except Exception as e:
        print(f"Error {filename} page {page_num}: {e}")
        return None


results = []
save_interval = 15
file_count = 0
pdf_files = [f for f in os.listdir(pdf_folder) if f.lower().endswith(".pdf")]
total_files = len(pdf_files)
start_time = time.time()

for filename in pdf_files:
    file_count += 1
    print(f"\nProcessing file {file_count}/{total_files}: {filename}")

    full_path = os.path.join(pdf_folder, filename)
    pdf_reader = PdfReader(full_path)
    num_pages = len(pdf_reader.pages)

    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = []
        for page_num in range(1, num_pages + 1):
            print(f" page {page_num}/{num_pages} for processing")
            futures.append(executor.submit(process_page, filename, full_path, page_num))

        for i, future in enumerate(as_completed(futures), 1):
            res = future.result()
            if res:
                results.append(res)
            print(f" {i}/{num_pages} pages for file {filename}")

    elapsed = time.time() - start_time
    avg_time_per_file = elapsed / file_count
    remaining_files = total_files - file_count
    est_remaining = remaining_files * avg_time_per_file
    print(f"Completed file {file_count}/{total_files} ({filename}) in {elapsed:.1f} sec")
    print(f"time: {est_remaining/60:.2f} minutes")

    if file_count % save_interval == 0 or file_count == total_files:
        df = pd.DataFrame(results)
        csv_name = f"DF{file_count}.csv"
        df.to_csv(csv_name, index=False)
        print(f"Saved results to {csv_name}")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Processing file 1/1: 02.pdf
 page 1/328 for processing
 page 2/328 for processing
 page 3/328 for processing
 page 4/328 for processing
 page 5/328 for processing
 page 6/328 for processing
 page 7/328 for processing
 page 8/328 for processing
 page 9/328 for processing
 page 10/328 for processing
 page 11/328 for processing
 page 12/328 for processing
 page 13/328 for processing
 page 14/328 for processing
 page 15/328 for processing
 page 16/328 for processing
 page 17/328 for processing
 page 18/328 for processing
 page 19/328 for processing
 page 20/328 for processing
 page 21/328 for processing
 page 22/328 for processing
 page 23/328 for processing
 page 24/328 for processing
 page 25/328 for processing
 page 26/328 for processing
 page 27/328 for processing
 page 28/328 for processing
 page 29/328 for processing
 page 30/328 for processing
 page 31/328 for processing
 page 32/328 for processing
 page 33/328 for processing
 page 34/328 for processing
 page 35/328 for processing


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 1/328 pages for file 02.pdf


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

 179/328 pages for file 02.pdf


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 180/328 pages for file 02.pdf


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 181/328 pages for file 02.pdf
